# this notebook is just for testing

In [ ]:
from platform import python_version
print(python_version())

In [ ]:
import os
import subprocess
import json
import numpy as np
import matplotlib.pyplot as pl

In [ ]:
def print_info(file):
    command = f"ffprobe -v error -select_streams v:0 -show_entries format:stream -print_format json {file}"
    cout = subprocess.run(command.split(), stdout=subprocess.PIPE, stderr=subprocess.STDOUT).stdout
    jout = json.loads(cout)
    print(jout)
            
def get_bitrate(file):
    command = f"ffprobe -v error -select_streams v:0 -show_entries format:stream -print_format json {file}"
    cout = subprocess.run(command.split(), stdout=subprocess.PIPE, stderr=subprocess.STDOUT).stdout
    jout = json.loads(cout)
    return int(jout["format"]["bit_rate"])

def get_vmaf(dist,ref):
    c_vmaf = f"ffmpeg -i {ref} -i {dist} -hide_banner -loglevel error\
            -lavfi \"[0:v]setpts=PTS-STARTPTS[ref];\
                    [1:v]setpts=PTS-STARTPTS[dist];\
                    [dist][ref]libvmaf=feature=name=psnr:log_path=config/vmaf_logs.json:log_fmt=json\" \
            -f null -" #|name=float_ssim|name=float_ms_ssim to compute the other metrics
    subprocess.call(c_vmaf, shell=True)
    with open("config/temp_vmaf_logs.json", 'r') as r: #extract quality and rate values
        i_data = json.load(r)
    return i_data["pooled_metrics"]["vmaf"]["mean"]

f_ref = "test.y4m"
f_dist = "test/"
print_info("test.y4m")

plot = {"PLOT": False, "FX": False, "BF": False, "LG": False, "CF": False, "TARGET": "rate"}

if plot["PLOT"]:
    my_dpi = 96
    pl.figure(figsize=(860/my_dpi, 540/my_dpi), dpi=my_dpi)

    if plot["FX"]:
        folder = "tests_vids/testAV1/"
        rate = np.zeros(len(os.listdir(folder)))
        dist = np.zeros(len(os.listdir(folder)))
        i = 0
        for vid in sorted(os.listdir(folder)):
            rate[i] = get_bitrate(folder + vid)
            dist[i] = get_vmaf(folder + vid,f_ref)
            i += 1
        print(rate)
        print(dist)
        pl.plot(rate/1000, dist, '-b', label="Fixed CRF")

    if plot["BF"]:
        folder = "tests_vids/testVP9/"
        rate = np.zeros(len(os.listdir(folder)))
        dist = np.zeros(len(os.listdir(folder)))
        i = 0
        for vid in sorted(os.listdir(folder)):
            rate[i] = get_bitrate(folder + vid)
            dist[i] = get_vmaf(folder + vid,f_ref)
            i += 1
        print(rate)
        print(dist)
        pl.plot(rate/1000, dist, '-g', label="Single shot")

    if plot["LG"]:
        folder = "tests_vids/OPT_vids/"+f_dist+"/lg_"+plot["TARGET"]+"/"
        rate = np.zeros(len(os.listdir(folder)))
        dist = np.zeros(len(os.listdir(folder)))
        i = 0
        for vid in sorted(os.listdir(folder)):
            rate[i] = get_bitrate(folder + vid)
            dist[i] = get_vmaf(folder + vid,f_ref)
            i += 1
        print(rate)
        print(dist)
        pl.plot(rate/1000, dist, '-c', label="OPT lg")

    if plot["CF"]:
        folder = "tests_vids/OPT_vids/"+f_dist+"/cf_"+plot["TARGET"]+"/"
        rate = np.zeros(len(os.listdir(folder)))
        dist = np.zeros(len(os.listdir(folder)))
        i = 0
        for vid in sorted(os.listdir(folder)):
            rate[i] = get_bitrate(folder + vid)
            dist[i] = get_vmaf(folder + vid,f_ref)
            i += 1
        print(rate)
        print(dist)
        pl.plot(rate/1000, dist, '-r', label="OPT cf")

    pl.grid(True)
    pl.title("RQ \"bar\" - AVC 1080p")
    pl.ylabel("Quality (VMAF)")
    pl.xlabel("Bitrate (Kbit/s)")
    pl.legend(loc="lower right")
    pl.figure()


In [ ]:
plot = {"a": False, "b": False, "c": False, "d": False}

my_dpi = 96
pl.figure(figsize=(860/my_dpi, 540/my_dpi), dpi=my_dpi)

if plot["a"]:
    rate = np.array([])
    dist = np.array([])
    pl.plot(rate/1000, dist, '-b', label="H.264")

if plot["b"]:
    rate = np.array([])
    dist = np.array([])
    pl.plot(rate/1000, dist, '-g', label="H.265")

if plot["c"]:
    rate = np.array([])
    dist = np.array([])
    pl.plot(rate/1000, dist, '-c', label="VP9")

if plot["d"]:
    rate = np.array([])
    dist = np.array([])
    pl.plot(rate/1000, dist, '-r', label="AV1")

bpl.grid(True)
pl.xlim(xmin=0)
pl.title("RQ \"test\" - codec comparison")
pl.ylabel("Quality (VMAF)")
pl.xlabel("Bitrate (Kbit/s)")
pl.legend(loc="lower right")
pl.figure()

In [ ]:
def perframe_vmaf(dist,ref):
    c_vmaf = f"ffmpeg -i {ref} -i {dist} -hide_banner -loglevel error\
            -lavfi \"[0:v]setpts=PTS-STARTPTS[ref];\
                    [1:v]setpts=PTS-STARTPTS[dist];\
                    [dist][ref]libvmaf=feature=name=psnr:log_path=config/vmaf_logs.json:log_fmt=json\" \
            -f null -" #|name=float_ssim|name=float_ms_ssim to compute the other metrics
    subprocess.call(c_vmaf, shell=True)
    print("-assessed= " + dist)
    with open("config/vmaf_logs4.json", 'r') as r: #extract quality and rate values
        i_data = json.load(r)
    return i_data["pooled_metrics"]["vmaf"]["mean"]

my_dpi = 96
pl.figure(figsize=(1080/my_dpi, 320/my_dpi), dpi=my_dpi)

window = 10
f_ref = "test.y4m"
f_refs = ["test/shot_0.y4m","test/shot_1.y4m",
         "test/shot_2.y4m","test/shot_3.y4m","test/shot_4.y4m"]
f_dist = "test.mp4"
f_dists = ["tests_vids/temp_encoded/0/32_AVC.mp4",
                  "tests_vids/temp_encoded/1/32_AVC.mp4",
                  "tests_vids/temp_encoded/2/28_AVC.mp4",
                  "tests_vids/temp_encoded/3/28_AVC.mp4",
                  "tests_vids/temp_encoded/4/32_AVC.mp4"]
print(perframe_vmaf(f_dist,f_ref))
ys = np.empty(0)
for i in range(5):
    print(perframe_vmaf(f_dists[i],f_refs[i]))
#pl.plot(x, ys, "-b", label="on each shot separately")

# siti VQEG

In [ ]:
siti-tools /home/ubuntu/dynopt2/tests_vids/shots_10sec/scene0.y4m > /home/ubuntu/dynopt2/tests_vids/shots_10sec/SITIscene0.y4m

In [ ]:
folder = "test/SITI_scores/"
num_sources = len(os.listdir(folder))
si = np.zeros(num_sources)
ti = np.zeros(num_sources)
source = np.empty(num_sources,dtype=object)

file_index = 0
for file in sorted(os.listdir(folder)):
    with open(folder + file, 'r') as f:
        data = json.load(f)
        si[file_index] = np.mean(data["si"])
        ti[file_index] = np.mean(data["ti"])
        source[file_index] = data["input_file"]
    file_index += 1

my_dpi = 96
pl.figure(figsize=(860/my_dpi, 540/my_dpi), dpi=my_dpi)
pl.plot(si,ti,'o',color="#002b49")
pl.xlim(xmin=0,xmax=60)
pl.ylim(ymin=0,ymax=16)
pl.title("Mean SI/TI Matrix")
pl.ylabel("mean TI")
pl.xlabel("mean SI")
pl.figure()

# Encode

In [ ]:
import subprocess
command = f"ffmpeg -f concat -safe 0 -i /home/ubuntu/video_dynopt/cuts.txt \
    -pix_fmt yuv420p -r 30 tests_vids/srcRAW_4K/bar_dinner.y4m"
subprocess.call(command, shell=True)